In [1]:
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import cell2location

from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42 # enables correct plotting of text for PDFs

Load trained model. (Consider retrain using filtered features.)

In [2]:
results_folder = '/data/gpfs/projects/punim0613/JiaDong/PhiSpace/'
# create paths and names to results folders for reference regression and cell2location models
ref_run_name = f'{results_folder}output/Case3/cell2location/reference_signatures'

adata_ref = sc.read(
    f'{results_folder}data/LungRef/AzimuthLung2.0_sce_0.1sub_selectFeat.h5ad'
)

Only considering the two last: ['.1sub_selectFeat', '.h5ad'].
Only considering the two last: ['.1sub_selectFeat', '.h5ad'].


In [3]:
# prepare anndata for the regression model
cell2location.models.RegressionModel.setup_anndata(adata=adata_ref,
                        # 10X reaction / sample / batch
                        # batch_key='Sample',
                        # cell type, covariate used for constructing signatures
                        labels_key='ann_finest_level',
                        # multiplicative technical effects (platform, 3' vs 5', donor effect)
                        # categorical_covariate_keys=['Method']
                       )

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
/data/projects/punim0613/JiaDong/miniconda3/envs/cell2location/lib/python3.10/site-packages/scvi/data/fields/_layer_field.py:115: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  _verify_and_correct_data_format(adata, self.attr_name, self.attr_key)


In [4]:
# create the regression model
from cell2location.models import RegressionModel
mod = RegressionModel(adata_ref)

# view anndata_setup as a sanity check
mod.view_anndata_setup()

Anndata setup with scvi-tools version 1.1.2.

Setup via `RegressionModel.setup_anndata` with arguments:

{
│   'layer': None,
│   'batch_key': None,
│   'labels_key': 'ann_finest_level',
│   'categorical_covariate_keys': None,
│   'continuous_covariate_keys': None
}

         Summary Statistics         
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃     Summary Stat Key     ┃ Value ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│         n_batch          │   1   │
│         n_cells          │ 58759 │
│ n_extra_categorical_covs │   0   │
│ n_extra_continuous_covs  │   0   │
│         n_labels         │  61   │
│          n_vars          │ 4613  │
└──────────────────────────┴───────┘

               Data Registry                
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Registry Key ┃    scvi-tools Location    ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      X       │          adata.X          │
│    batch     │ adata.obs['_scvi_batch']  │
│    ind_x     │   adata.obs['_indices']   │
│    labels    │ adata.obs['_scvi_labels'] │
└──────────────┴───────────────────────────┘

                     batch State Registry                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃     Source Location      ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['_scvi_batch'] │     0      │          0          │
└──────────────────────────┴────────────┴─────────────────────┘

                                 labels State Registry                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃        Source Location        ┃          Categories           ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['ann_finest_level'] │         Basal resting         │          0          │
│                               │          Suprabasal           │          1          │
│                               │         Hillock-like          │          2          │
│                               │         Deuterosomal          │          3          │
│                               │     Multiciliated (nasal)     │          4          │
│                               │   Multiciliated (non-nasal)   │          5          │
│                               │       Club (non-nasal)        │          6          │
│                               │         Club (nasal)          │          7          │
│                               │        Goblet (nasal)         │          8          │
│                               │      Goblet (bronchial)       │          9          │
│                               │     Goblet (subsegmental)     │         10          │
│                               │              AT0              │         11          │
│                               │       pre-TB secretory        │         12          │
│                               │           Ionocyte            │         13          │
│                               │             Tuft              │         14          │
│                               │        Neuroendocrine         │         15          │
│                               │      SMG serous (nasal)       │         16          │
│                               │    SMG serous (bronchial)     │         17          │
│                               │          SMG mucous           │         18          │
│                               │           SMG duct            │         19          │
│                               │              AT1              │         20          │
│                               │              AT2              │         21          │
│                               │       AT2 proliferating       │         22          │
│                               │          EC arterial          │         23          │
│                               │     EC aerocyte capillary     │         24          │
│                               │     EC general capillary      │         25          │
│                               │      EC venous systemic       │         26          │
│                               │      EC venous pulmonary      │         27          │
│                               │      Lymphatic EC mature      │         28          │
│                               │  Lymphatic EC proliferating   │         29          │
│                               │ Lymphatic EC differentiating  │         30          │
│                               │   Peribronchial fibroblasts   │         31          │
│                               │    Adventitial fibroblasts    │         32          │
│                               │     Alveolar fibroblasts      │         33          │
│                               │           Pericytes           │         34          │
│                               │    Subpleural fibroblasts     │         35          │
│                               │        Myofibroblasts         │         36          │
│                               │         Smooth muscle         │         37          │
│                               │     Smooth muscle FAM83D+     │         38          │
│                               │ SM activated stress response  │         39          │
│                               │          Mesothelium          │         40          │
│                               │   Hema

In [5]:
mod.train(max_epochs=250)

/data/projects/punim0613/JiaDong/miniconda3/envs/cell2location/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /data/projects/punim0613/JiaDong/miniconda3/envs/cel ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/data/projects/punim0613/JiaDong/miniconda3/envs/cell2location/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /data/projects/punim0613/JiaDong/miniconda3/envs/cel ...
/data/projects/punim0613/JiaDong/miniconda3/envs/cell2location/lib/python3.10/site-packa

Epoch 250/250: 100%|██████████| 250/250 [34:28<00:00,  8.25s/it, v_num=1, elbo_train=1.96e+8]

`Trainer.fit` stopped: `max_epochs=250` reached.


Epoch 250/250: 100%|██████████| 250/250 [34:28<00:00,  8.28s/it, v_num=1, elbo_train=1.96e+8]


In [6]:
# In this section, we export the estimated cell abundance (summary of the posterior distribution).
adata_ref = mod.export_posterior(
    adata_ref, sample_kwargs={'num_samples': 1000, 'batch_size': 2500}
)

/data/projects/punim0613/JiaDong/miniconda3/envs/cell2location/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /data/projects/punim0613/JiaDong/miniconda3/envs/cel ...


Sampling global variables, sample: 100%|██████████| 999/999 [00:06<00:00, 144.33it/s]


In [7]:
# Save model
mod.save(f"{ref_run_name}", overwrite=True)

# Save anndata object with results
adata_file = f"{ref_run_name}/sc.h5ad"
adata_ref.write(adata_file)